**Connect Through SQLite Database**

In [1]:
import sqlite3
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
#Establish a connection
connection = sqlite3.connect('AI_Symptomps_Note.db')
# Create a cursor object
cursor = connection.cursor()

In [3]:
# Fetch data from the database
query = "SELECT * FROM df_filtered"
data = pd.read_sql_query(query, connection)
# Display the first few rows of the dataframe
print(data.head())

                         search_term  \
0              เสมหะไหลลงคอ, มีเสมหะ   
1                    บวม, ปวดข้อเท้า   
2  หายใจมีเสียงวี๊ด, หายใจหอบเหนื่อย   
3                         ปวดหัว, ชา   
4                            ไข้, ไอ   

                                 symptoms_note_clean  
0                  ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว  
1  ตำแหน่ง บวมที่ข้อ เช่น ข้อเข่า ข้อเท้า, ระดับ ...  
2  ระดับ เหนื่อยเล็กน้อย เหนื่อยง่ายขึ้นเวลาออกแร...  
3  ระดับ ปวดเล็กน้อย ส่งผลต่อการดำเนินกิจวัตรประจ...  
4                        3 วัน, ระดับ ไข้ปานกลาง (38  


In [4]:
Symptoms = data
Symptoms

,search_term,symptoms_note_clean
0,"เสมหะไหลลงคอ, มีเสมหะ",ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว
1,"บวม, ปวดข้อเท้า","ตำแหน่ง บวมที่ข้อ เช่น ข้อเข่า ข้อเท้า, ระดับ ..."
2,"หายใจมีเสียงวี๊ด, หายใจหอบเหนื่อย",ระดับ เหนื่อยเล็กน้อย เหนื่อยง่ายขึ้นเวลาออกแร...
3,"ปวดหัว, ชา",ระดับ ปวดเล็กน้อย ส่งผลต่อการดำเนินกิจวัตรประจ...
4,"ไข้, ไอ","3 วัน, ระดับ ไข้ปานกลาง (38"
...,...,...
107,ไข้,"3 วัน, ระดับ ไข้ต่ำ (37"
108,"ตัวร้อน, ไข้","7 วัน, ระดับ ไข้ปานกลาง (38"
109,"ร้อนวูบวาบ, ตัวร้อน","3 วัน, ระดับ ไข้ต่ำ (37"
110,ไข้,"3 วัน, ระดับ ไข้ต่ำ (37"


In [5]:
Symptoms.head()

,search_term,symptoms_note_clean
0,"เสมหะไหลลงคอ, มีเสมหะ",ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว
1,"บวม, ปวดข้อเท้า","ตำแหน่ง บวมที่ข้อ เช่น ข้อเข่า ข้อเท้า, ระดับ ..."
2,"หายใจมีเสียงวี๊ด, หายใจหอบเหนื่อย",ระดับ เหนื่อยเล็กน้อย เหนื่อยง่ายขึ้นเวลาออกแร...
3,"ปวดหัว, ชา",ระดับ ปวดเล็กน้อย ส่งผลต่อการดำเนินกิจวัตรประจ...
4,"ไข้, ไอ","3 วัน, ระดับ ไข้ปานกลาง (38"


# **Association Rules** For extracting relevant symptoms and match them with symptom set

In [6]:
def preprocess_symptoms(Symptoms):
    # Handle NaN values in symptoms_note
    Symptoms['symptoms_note_clean'] = Symptoms['symptoms_note_clean'].fillna('')
    
    # Split symptoms into list
    Symptoms['symptoms_list'] = Symptoms['symptoms_note_clean'].str.split(',')
    
    # Get unique symptoms - filter out empty strings and None values
    all_symptoms = set()
    for symptoms in Symptoms['symptoms_list']:
        if isinstance(symptoms, list):
            # Strip whitespace and filter out empty strings
            cleaned_symptoms = [s.strip() for s in symptoms if s and s.strip()]
            all_symptoms.update(cleaned_symptoms)
    
    # Create one-hot vectors manually
    symptoms_encoded = pd.DataFrame(0, index=Symptoms.index, columns=list(all_symptoms))
    
    for idx, symptoms in enumerate(Symptoms['symptoms_list']):
        if isinstance(symptoms, list):
            for symptom in symptoms:
                # Only process non-empty symptoms
                if symptom and symptom.strip():
                    symptoms_encoded.loc[idx, symptom.strip()] = 1
    
    # Handle any remaining NaN values
    symptoms_encoded = symptoms_encoded.fillna(0)
    
    # Apply min-max scaling
    scaler = MinMaxScaler()
    symptoms_scaled = pd.DataFrame(
        scaler.fit_transform(symptoms_encoded),
        columns=symptoms_encoded.columns
    )
    
    return symptoms_scaled

In [7]:
def generate_association_rules(symptoms_encoded, min_support=0.2, min_confidence=0.8):
    # Find frequent itemsets
    frequent_itemsets = apriori(symptoms_encoded, 
                              min_support=min_support, 
                              use_colnames=True)
    
    # Generate rules
    rules = association_rules(frequent_itemsets, 
                            metric="confidence", 
                            min_threshold=min_confidence)
    
    return rules

In [8]:
def select_features(rules, symptoms_encoded, top_n=10):
    # Select features based on lift score
    important_features = rules.nlargest(top_n, 'lift')
    selected_features = set()
    
    for _, row in important_features.iterrows():
        selected_features.update(row['antecedents'])
        selected_features.update(row['consequents'])
    
    return symptoms_encoded[list(selected_features)]

In [9]:
# Main implementation
def implement_symptom_analysis(Symptoms):
    # Preprocess the data
    symptoms_encoded = preprocess_symptoms(Symptoms).astype(bool)
    
    # Generate association rules
    rules = generate_association_rules(symptoms_encoded)
    
    # Select important features
    selected_features = select_features(rules, symptoms_encoded)
    
    print("Shape of encoded symptoms:", symptoms_encoded.shape)
    print("\nNumber of association rules generated:", len(rules))
    print("\nTop features by lift score:", list(selected_features.columns))
    
    return rules, selected_features

In [10]:
# Example usage
rules, selected_features = implement_symptom_analysis(Symptoms)

Shape of encoded symptoms: (112, 91)

Number of association rules generated: 1

Top features by lift score: ['ระดับ ไข้ต่ำ (37', '3 วัน']


c:\Users\pc\anaconda3\envs\Symptoms_Recoomendation\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


In [11]:
def find_associated_symptoms(input_symptom, Symptoms):
    """
    Find associated symptoms for a specific symptom.
    Returns a list of associated symptoms (strings). Empty list if none found.
    """
    # Clean and prepare the data
    symptoms_pairs = Symptoms[['symptoms_note_clean', 'search_term']].dropna()

    # Find rows that match the input symptom in either column (case-insensitive)
    mask = (
        symptoms_pairs['search_term'].str.contains(input_symptom, case=False, na=False) |
        symptoms_pairs['symptoms_note_clean'].str.contains(input_symptom, case=False, na=False)
    )
    matching_rows = symptoms_pairs[mask]

    if matching_rows.empty:
        return []

    # Collect associated symptoms from all matching rows
    associated = []
    for _, row in matching_rows.iterrows():
        raw = str(row['symptoms_note_clean'])
        parts = [p.strip() for p in raw.split(',') if p.strip()]
        associated.extend(parts)

    # Remove the input symptom itself (case-insensitive match)
    input_lower = input_symptom.lower()
    unique_associated = []
    seen = set()
    for s in associated:
        if s is None:
            continue
        s_clean = s.strip()
        if s_clean == "":
            continue
        if input_lower != s_clean.lower() and s_clean.lower() not in seen:
            unique_associated.append(s_clean)
            seen.add(s_clean.lower())

    print(f"\nResults for '{input_symptom}':")
    print("Associated symptoms:")
    for symptom in unique_associated:
        print(f"- {symptom}")
    print("-" * 50)

    return unique_associated

In [12]:
print("\nCase 1: ไอ, มีเสมหะคัดจมูก")
case1 = find_associated_symptoms('ไอ, มีเสมหะคัดจมูก', Symptoms)


Case 1: ไอ, มีเสมหะคัดจมูก

Results for 'ไอ, มีเสมหะคัดจมูก':
Associated symptoms:
- 3 สัปดาห์
- ลักษณะ ไอไม่มีเสมหะ ไอแห้งๆ หรือไอเสมหะสีขาว
--------------------------------------------------


In [13]:
print("\nCase 2: ปวดเมื่อยกล้ามเนื้อทั่วๆ, ปวดคอปวดหัวไหล่")
case2 = find_associated_symptoms('ปวดเมื่อยกล้ามเนื้อทั่วๆ, ปวดคอปวดหัวไหล่', Symptoms)


Case 2: ปวดเมื่อยกล้ามเนื้อทั่วๆ, ปวดคอปวดหัวไหล่

Results for 'ปวดเมื่อยกล้ามเนื้อทั่วๆ, ปวดคอปวดหัวไหล่':
Associated symptoms:
- ร้าว ร้าวไปศีรษะ
- ชามือหรือแขน ไม่ชา
- อ่อนแรง ไม่อ่อนแรง
- ระดับ ปวดเล็กน้อย ส่งผลต่อการดำเนินกิจวัตรประจำวันบ้าง
--------------------------------------------------


In [14]:
print("\nCase 3: ปวดตา, ตาบวม")
case3 = find_associated_symptoms('ปวดตา, ตาบวม', Symptoms)


Case 3: ปวดตา, ตาบวม

Results for 'ปวดตา, ตาบวม':
Associated symptoms:
- ระดับ ปวดเล็กน้อย ส่งผลต่อการดำเนินกิจวัตรประจำวันบ้าง
- ข้าง ตาเดียว
--------------------------------------------------


In [15]:
print("\nCase 4: back pain")
case4 = find_associated_symptoms('back pain', Symptoms)


Case 4: back pain

Results for 'back pain':
Associated symptoms:
- Location midline lower back
- pain to the thigh
- numbness yes
- weakness no
- Severity Moderate
- pain none
- arm or hand numbness no
- weakness yes
- Severity Moderate unable to continue with your work
- Location lower paraspinal (muscles closely surround the spine)
--------------------------------------------------


In [16]:
print("\nCase 5: Fever")
case5 = find_associated_symptoms('Fever', Symptoms)


Case 5: Fever

Results for 'Fever':
Associated symptoms:
- 3weeks
- types Yellowish/greenish phlegm
- Severity Moderate
--------------------------------------------------


In [17]:
print("\nCase 6: stomachache, diarrhea")
case6 = find_associated_symptoms('stomachache, diarrhea', Symptoms)


Case 6: stomachache, diarrhea

Results for 'stomachache, diarrhea':
Associated symptoms:
- location All over
- duration less than a day
- severity mild
- characteristic loose/watery
--------------------------------------------------
